In [1]:
!pip install --quiet --upgrade langchain langchain-google-genai cache-pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 10.7 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import userdata

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
print(GOOGLE_API_KEY)  # 출력 시 None이면 비밀 키를 불러오지 못한 상태

In [4]:
import requests
import pandas as pd
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema import HumanMessage

In [5]:
dg_blog_1 = pd.read_csv('/content/drive/MyDrive/보아즈 시각화 미니프로젝트 1차/크롤링 데이터/블로그_대구치맥페스티벌.csv')
dg_blog_2 = pd.read_csv('/content/drive/MyDrive/보아즈 시각화 미니프로젝트 1차/크롤링 데이터/블로그_대구서문시장.csv')

In [6]:
dg_youtube_1 = pd.read_csv('/content/drive/MyDrive/보아즈 시각화 미니프로젝트 1차/크롤링 데이터/대구 치맥 페스티벌_youtube_comments_with_attributes.csv')
dg_youtube_2 = pd.read_csv('/content/drive/MyDrive/보아즈 시각화 미니프로젝트 1차/크롤링 데이터/서문시장_youtube_comments_with_attributes.csv')

In [7]:
ys_blog_1 = pd.read_csv('/content/drive/MyDrive/보아즈 시각화 미니프로젝트 1차/크롤링 데이터/블로그_예산맥주페스티벌.csv')
ys_blog_2 = pd.read_csv('/content/drive/MyDrive/보아즈 시각화 미니프로젝트 1차/크롤링 데이터/블로그_예산시장.csv')

In [8]:
ys_youtube_1 = pd.read_csv('/content/drive/MyDrive/보아즈 시각화 미니프로젝트 1차/크롤링 데이터/예산 맥주 페스티벌_youtube_comments_with_attributes.csv')
ys_youtube_2 = pd.read_csv('/content/drive/MyDrive/보아즈 시각화 미니프로젝트 1차/크롤링 데이터/예산 시장_youtube_comments_with_attributes.csv')

In [9]:
dg_blog = pd.concat([dg_blog_1,dg_blog_2])
dg_youtube = pd.concat([dg_youtube_1,dg_youtube_2])
ys_blog = pd.concat([ys_blog_1,ys_blog_2])
ys_youtube = pd.concat([ys_youtube_1,ys_youtube_2])

In [10]:
dg_blog = dg_blog.drop(columns = ['title', 'author', 'datetime'])
dg_youtube = dg_youtube[['comment']]
ys_blog = ys_blog.drop(columns = ['title', 'author', 'datetime'])
ys_youtube = ys_youtube[['comment']]

In [11]:
dg_blog = dg_blog.dropna(subset=['contents'])
dg_blog = dg_blog.reset_index()
dg_blog.drop(columns = ['index'],inplace = True)

ys_blog = ys_blog.dropna(subset=['contents'])
ys_blog = ys_blog.reset_index()
ys_blog.drop(columns = ['index'],inplace = True)

dg_youtube = dg_youtube.dropna(subset=['comment'])
dg_youtube = dg_youtube.reset_index()
dg_youtube.drop(columns = ['index'],inplace = True)

dg_youtube = dg_youtube.dropna(subset=['comment'])
dg_youtube = dg_youtube.reset_index()
dg_youtube.drop(columns = ['index'],inplace = True)

In [12]:
import re

def clean_text(text):
    # 1. 줄바꿈과 공백 정리
    text = re.sub(r'\n+', '\n', text.strip())  # 여러 개의 연속된 줄바꿈은 하나로
    text = re.sub(r'[ ]{2,}', ' ', text)  # 여러 공백을 하나로

    # 2. URL 제거
    text = re.sub(r'http\S+|www\S+|blog.naver.com\S+', '', text)

    # 3. 특수 문자 및 이모티콘 제거
    text = re.sub(r'[^가-힣0-9a-zA-Z.,?!\s]', '', text)  # 한글, 영어, 숫자, 일부 문장부호만 허용

    # 4. 불필요한 메타 정보 제거 (예: 날짜, 시간, 복사, 기타)
    text = re.sub(r'\d{1,2}[:.]\d{2}(:\d{2})?', '', text)  # 시간 제거 (예: 2:47)
    text = re.sub(r'\d{4}\.\s?\d{1,2}\.\s?\d{1,2}\.', '', text)  # 날짜 제거 (예: 2023. 9. 4.)
    text = re.sub(r'© NAVER Corp.', '', text)
    text = re.sub(r'URL 복사 이웃추가 본문 기타 기능', '', text)

    # 5. 양쪽 공백 제거
    text = text.strip()

    return text



In [14]:
# 전처리
dg_blog['contents'] = dg_blog['contents'].apply(lambda x: clean_text(x))
dg_youtube['comment'] = dg_youtube['comment'].apply(lambda x: clean_text(x))

ys_blog['contents'] = ys_blog['contents'].apply(lambda x: clean_text(x))
ys_youtube['comment'] = ys_youtube['comment'].apply(lambda x: clean_text(x))


In [19]:
import time
from transformers import GPT2TokenizerFast  # 토큰화 도구 사용

# GPT-2 토크나이저 초기화 (Google Gemini와 호환되는 기본 토큰화 방식 사용)
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

# LLM 초기화
llm = ChatGoogleGenerativeAI(
    model="gemini-pro",
    google_api_key=GOOGLE_API_KEY
)

def split_text_by_token_limit(text, token_limit=32000):
    """텍스트를 주어진 토큰 제한에 따라 분할합니다."""
    tokens = tokenizer.encode(text)
    for i in range(0, len(tokens), token_limit):
        yield tokenizer.decode(tokens[i:i + token_limit])

def get_sentiment_with_limit(text, delay=1, token_limit=32000):
    """감정 분석 함수로, 토큰 제한에 따라 텍스트를 분할하여 처리합니다."""
    sentiments = []

    for part in split_text_by_token_limit(text, token_limit):
        prompt = HumanMessage(content=f"Analyze the sentiment of the following text: '{part}'. Return 'positive', 'negative', or 'neutral' based on the sentiment analysis.")
        response = llm([prompt])
        sentiment = response.content.strip().lower()

        if 'positive' in sentiment:
            sentiments.append('positive')
        elif 'negative' in sentiment:
            sentiments.append('negative')
        else:
            sentiments.append('neutral')

        time.sleep(delay)  # 호출 간격 조정

    # 전체 감정을 종합하여 대다수 감정을 반환
    return max(set(sentiments), key=sentiments.count)

def get_positive_negative_keywords(text, delay=1):
    """텍스트에서 긍정 키워드 5개와 부정 키워드 5개를 추출합니다."""
    # 키워드 추출
    prompt = HumanMessage(content=f"Extract 10 keywords from the text: '{text}'. Return them separated by commas.")
    response = llm([prompt])
    time.sleep(delay)

    keywords = response.content.strip().lower().split(", ")

    positive_keywords = []
    negative_keywords = []

    # 각 키워드에 대해 감정 분석 수행
    for keyword in keywords:
        sentiment_prompt = HumanMessage(content=f"Analyze the sentiment of the keyword: '{keyword}'. Return 'positive' or 'negative'.")
        sentiment_response = llm([sentiment_prompt])
        sentiment = sentiment_response.content.strip().lower()

        if 'positive' in sentiment:
            positive_keywords.append(keyword)
        elif 'negative' in sentiment:
            negative_keywords.append(keyword)

        time.sleep(delay)  # 호출 간격 조정

    # 상위 5개씩 추출
    return positive_keywords[:5], negative_keywords[:5]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [21]:
from google.colab import files

# Apply sentiment analysis to the dataset
dg_blog['Sentiment'] = dg_blog['contents'].apply(lambda x: get_sentiment_with_limit(x, delay=10, token_limit=32000))
# positive_keyword와 negative_keyword 열 생성
dg_blog[['positive_keyword', 'negative_keyword']] = dg_blog['contents'].apply(
    lambda x: pd.Series(get_positive_negative_keywords(x, delay=10))
)
# 데이터프레임을 CSV로 저장
dg_blog.to_csv('dg_blog.csv', index=False)  # 데이터프레임의 인덱스를 제외하고 저장
# 로컬에 다운로드
files.download('dg_blog.csv')

ResourceExhausted: 429 Resource has been exhausted (e.g. check quota).

In [ ]:
dg_youtube['Sentiment'] = dg_blog['comment'].apply(lambda x: get_sentiment_with_limit(x, delay=5, token_limit=32000))
# positive_keyword와 negative_keyword 열 생성
dg_youtube[['positive_keyword', 'negative_keyword']] = dg_blog['comment'].apply(
    lambda x: pd.Series(get_positive_negative_keywords(x, delay=5))
)

# 데이터프레임을 CSV로 저장
dg_youtube.to_csv('dg_youtube.csv', index=False)  # 데이터프레임의 인덱스를 제외하고 저장

files.download('dg_youtube.csv')

In [ ]:
ys_blog['Sentiment'] = ys_blog['contents'].apply(lambda x: get_sentiment_with_limit(x, delay=5, token_limit=32000))
# positive_keyword와 negative_keyword 열 생성
ys_blog[['positive_keyword', 'negative_keyword']] = ys_blog['contents'].apply(
    lambda x: pd.Series(get_positive_negative_keywords(x, delay=5))
)

# 데이터프레임을 CSV로 저장
ys_blog.to_csv('ys_blog.csv', index=False)  # 데이터프레임의 인덱스를 제외하고 저장

files.download('ys_blog.csv')

In [ ]:
ys_youtube['Sentiment'] = ys_blog['comment'].apply(lambda x: get_sentiment_with_limit(x, delay=5, token_limit=32000))
# positive_keyword와 negative_keyword 열 생성
ys_youtube[['positive_keyword', 'negative_keyword']] = ys_blog['comment'].apply(
    lambda x: pd.Series(get_positive_negative_keywords(x, delay=5))
)

# 데이터프레임을 CSV로 저장
ys_youtube.to_csv('ys_youtube.csv', index=False)  # 데이터프레임의 인덱스를 제외하고 저장

files.download('ys_youtube.csv')